Conversion from .emd to .h5 format.

In [ ]:
# Conversion Function Definitions

from conversion_def import find_and_process_velox_files

In [ ]:
# Convert to HDF5
inputfolder_path= ""

find_and_process_velox_files(inputfolder_path)

Peakfinding using peakfinder8 from diffractem package. First open .h5 file in crystfel using .geom file that refers to correct mask file and find good peakfinding settings. Make sure there are enough friedel pairs for center beam finding and fitting. Don't put the mask in the same folder as the .h5 file since it is also .h5 format. If peakfinding has been run old data will be deleted.

In [9]:
# Peak Finder Function Definitions

from peakfinder_def import find_files_and_run_peakfinding

In [ ]:
# Peak Finder
inputfolder_path= ""

# Peakfinder settings:
threshold = 80
min_snr = 3
min_pix_count = 2
max_pix_count = 200
local_bg_radius = 9
min_res = 0
max_res = 1000
x0 = 512
y0 = 512  

##### Enter parameters above #####

find_files_and_run_peakfinding(inputfolder_path, x0, y0, threshold, min_snr, min_pix_count, max_pix_count, local_bg_radius, min_res, max_res)

Find the center beam spot center on all frames using friedel pairs and  interpolation.

In [11]:
# Find Center Beam Spot Center Function Definitions

from centerfinding_def import find_files_and_run_centerfinding

In [ ]:
# Find Center Beam Spot Center
tolerance = 25 # If you know that your beam center is close to the center of the detector you can reduce the tolerance to reduce the processing time
min_peaks = 30 # Minimum number of peaks in a frame to be considered
resolution_limit = 150 # Maximum resultuion of peaks to be considered
min_samples_fraction = 0.05  # Fraction of datapoints in the largest cluster to be accepted
x0 = 512
y0 = 512

inputfolder_path= ""

##### Enter parameters above #####

find_files_and_run_centerfinding(inputfolder_path, x0, y0, tolerance, min_peaks, resolution_limit, min_samples_fraction)

Improve the center beam center with a LOWESS fit

In [13]:
# Center Beam Drift Refinement Function Definitions

from centerrefinement_def import find_files_and_run_centerrefinement


In [ ]:
# Center Beam Drift Refinement
tolerance = 6 # If you know that your beam center is close to the center of the detector you can reduce the tolerance to reduce the processing time
min_peaks = 30 # Minimum number of peaks in frame to be used for refinement
resolution_limit = 150 # Maximum resolution of peaks to be considered for refinement
max_iterations = 10 # Maximum number of iterations for refinement
convergence_threshold = 0.05 # Refinement is stopped when the deviation of the LOWESS fit does not exceed the threshold (given in pixels)

inputfolder_path= ""

##### Enter your parameters above #####

find_files_and_run_centerrefinement(inputfolder_path, tolerance, min_peaks, resolution_limit, max_iterations, convergence_threshold)

And now to find the indexing using xGandalf (5x5=25 output streams as default)

In [1]:
# Indexing Function Definitions

from gandalf_iterator import gandalf_iterator

In [ ]:
# Indexing (To speed up process make sure output stream is in wsl folder)
geomfile_path = "/home/buster/UOX1/UOX.geom"
cellfile_path = "/home/buster/UOX1/UOX.cell"
input_path = "/home/buster/UOX1"
output_dir = "/home/buster/UOX1"
output_file_base = "UOX1"

num_threads = 23
indexing_method = "xgandalf"
resolution_push = 0
integration_method = "rings"
int_radius = "4,5,8"
min_peaks = 15
xgandalf_tolerance = 0.02
xgandalf_sampling_pitch = 5
xgandalf_min_vector_length = 40
xgandalf_max_vector_length = 110
xgandalf_iterations = 1
tolerance = "5,5,5,5"
step= 0.01
layers= 2
##### Enter your parameters above #####

gandalf_iterator(geomfile_path, cellfile_path, input_path, output_file_base, output_dir, num_threads, indexing_method, resolution_push, integration_method, int_radius, min_peaks, xgandalf_tolerance, xgandalf_sampling_pitch, xgandalf_min_vector_length, xgandalf_max_vector_length, xgandalf_iterations, tolerance, step, layers)

Then based on RMSD between found peaks and indexed reflections combine the best indexings from xGandalf for each frame to one .stream file and then write a .sol from combinded .stream file.

In [1]:
# Find Best Results Function Definitions

from best_results_def import find_best_results, read_stream_write_sol

In [2]:
# Find Best Results
inputfolder_path = "/home/buster/UOX1/5x5"
output_path= "/home/buster/UOX1/5x5"
lattice = "oI" # Pearson Symbol read from .cell file first letter for lattice type and second for centering

##### Enter your parameters above #####

# find_best_results(inputfolder_path, output_path)

import os

output_file_path = os.path.join(inputfolder_path, "best_results_RMSD.stream")

read_stream_write_sol(output_file_path, lattice)

['/home/buster/UOX1/5x5/UOX1_-511.98_-512.02.stream', '/home/buster/UOX1/5x5/UOX1_-512.0_-512.01.stream', '/home/buster/UOX1/5x5/UOX1_-512.02_-512.01.stream', '/home/buster/UOX1/5x5/UOX1_-512.0_-511.98.stream', '/home/buster/UOX1/5x5/UOX1_-512.0_-511.99.stream', '/home/buster/UOX1/5x5/UOX1_-512.01_-512.01.stream', '/home/buster/UOX1/5x5/UOX1_-512.01_-511.98.stream', '/home/buster/UOX1/5x5/best_results_RCIS_1_2_1_-1_1_1.stream', '/home/buster/UOX1/5x5/UOX1_-512_-512.stream', '/home/buster/UOX1/5x5/UOX1_-512.02_-512.0.stream', '/home/buster/UOX1/5x5/best_results_RCIS_1_1_2_-1_1_1.stream', '/home/buster/UOX1/5x5/UOX1_-511.98_-512.0.stream', '/home/buster/UOX1/5x5/UOX1_-512.01_-512.0.stream', '/home/buster/UOX1/5x5/UOX1_-512.01_-512.02.stream', '/home/buster/UOX1/5x5/UOX1_-511.98_-511.99.stream', '/home/buster/UOX1/5x5/best_results_RCIS_1_3_2_-1_1_1.stream', '/home/buster/UOX1/5x5/UOX1_-511.98_-511.98.stream', '/home/buster/UOX1/5x5/UOX1_-512.02_-511.98.stream', '/home/buster/UOX1/5x5/UOX1

FileNotFoundError: [Errno 2] No such file or directory: '/home/buster/UOX1/5x5/best_results_RMSD.stream'

Integrate with best found ring for integration (From Fast Integration)

In [ ]:
# Full Integration Function Definitions

from full_int_def import run_indexamajig_with_frames

In [ ]:
# Full Integration
geomfile_path = ".geom"
cellfile_path = ".cell"
input_path = ""

x = -512
y = -512
num_threads = 23
indexing_method = "file"
resolution_push = 5
integration_method = "rings"
int_radius = "4,5,9"
min_peaks = 5
xgandalf_tolerance = 0.02
xgandalf_sampling_pitch = 5
xgandalf_min_vector_length = 40
xgandalf_max_vector_length = 110
xgandalf_iterations = 1
tolerance = "5,5,5,5"
overpredict = True

##### Enter your parameters above #####

run_indexamajig_with_frames(x, y, geomfile_path, cellfile_path, input_path, num_threads, indexing_method, resolution_push, integration_method, int_radius, min_peaks, xgandalf_tolerance, xgandalf_sampling_pitch, xgandalf_iterations, tolerance)

Merge integration results

In [ ]:
# Full Merge Function Definitions

from full_merge_def import merge_and_write_mtz

In [ ]:
# Merge
input_folder_path = ""
cellfile_path = ""
pointgroup = ""

##### Enter your parameters above #####

merge_and_write_mtz(input_folder_path, cellfile_path, pointgroup)

Refinement using REFMAC5

In [ ]:
# Full Refine Function Definitions

from full_ref_def import process_run_folders

In [ ]:
# Full Refine

run_number = 1
bins = 30

# Global variables for filenames
base_path = ""
pdb_filename = ".pdb"
mtz_orig_filename = "output.mtz"
output_filename = f"output_bins_{bins}.txt"

process_run_folders(base_path, run_number)